# 1. 데이터 로드

In [1]:
import os
import sys
import time
import random
import datetime
import requests
import pandas as pd
import numpy as np
import hashlib, hmac, base64
from itertools import combinations, permutations
from dtw import *
import json
import urllib.request
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
import statsmodels.api as sm
import pickle
from pytz import timezone
from difflib import SequenceMatcher

from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from collections import defaultdict
from pytrends.request import TrendReq
import nest_asyncio
from typing import List, Dict, Any



Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



# 2. API설정

In [2]:
from api_set import APIClient

# API 설정
from utils import get_secret
BASE_URL = get_secret("BASE_URL")
CUSTOMER_ID = get_secret("CUSTOMER_ID")
API_KEY = get_secret("API_KEY")
SECRET_KEY = get_secret("SECRET_KEY")
URI = get_secret("URI")
METHOD = get_secret("METHOD")
# API 클라이언트 인스턴스 생성
api_client = APIClient(BASE_URL, CUSTOMER_ID, API_KEY, SECRET_KEY,URI,METHOD)


# 3. 연관검색어 수집

In [3]:
# 키 로드
from utils import load_keywords 
keywords_data = load_keywords('main_keyword.json')

from utils import get_today_date
# 오늘의 날짜 가져오기
formatted_today, day = get_today_date()


# 결과 저장 폴더 생성
from utils import make_directory

make_directory('./data')
make_directory('./data/rl_srch')
make_directory(f'./data/rl_srch/{day}')  # 키워드별 연관검색어 리스트 저장

In [4]:

# 검색어 리스트와 결과 저장 경로 설정
srch_keyword = ['keyword_final']  
save_path = './data/rl_srch/'  
print(api_client.base_url)

https://api.searchad.naver.com


In [5]:


# nest_asyncio를 사용하여 이미 실행 중인 이벤트 루프와의 충돌을 방지합니다.
nest_asyncio.apply()

# 비동기 수집 함수를 여기에 정의하거나 import합니다.
from collect_keywords import collect_keywords

# 오늘 날짜를 YYYYMMDD 형식으로 생성합니다.
today_str = datetime.now().strftime("%y%m%d")

# 파일 저장 경로를 설정합니다. 'data/rl_srch/YYYYMMDD/collected_keywords.csv' 형식을 사용합니다.
save_path = os.path.join('data', 'rl_srch', today_str, 'collected_keywords.csv')

# 파일이 이미 존재하는지 확인합니다.
if not os.path.exists(save_path):
    # 파일이 존재하지 않는 경우, 비동기 수집 함수를 실행합니다.
    start = time.time()
    collected_keywords_data = await collect_keywords(srch_keyword, day)
    
    # 수집된 데이터를 DataFrame으로 변환합니다. (collect_keywords의 반환 값이 이미 DataFrame이라면 이 단계는 생략 가능합니다.)
    # collected_keywords_data = pd.DataFrame(collected_keywords_data)

    # 파일 저장 경로의 디렉토리가 존재하지 않는 경우 생성합니다.
    os.makedirs(os.path.dirname(save_path), exist_ok=True)

    # DataFrame을 CSV 파일로 저장합니다.
    collected_keywords_data.to_csv(save_path, index=False)
    
    print(f"Data saved to {save_path}")
    print('Elapsed time:', time.time() - start)
else:
    print(f"File already exists at {save_path}, skipping saving process.")


File already exists at data\rl_srch\240307\collected_keywords.csv, skipping saving process.


In [6]:
import pandas as pd
import os
from datetime import datetime

# 오늘 날짜를 YYYYMMDD 형식으로 생성합니다.
today_str = datetime.now().strftime("%y%m%d")

# 파일 저장 경로를 설정합니다. 'data/rl_srch/YYYYMMDD/collected_keywords.csv' 형식을 사용합니다.
save_path = os.path.join('data', 'rl_srch', today_str, 'collected_keywords.csv')

# 파일이 존재하는지 확인하고, 존재할 경우 데이터를 불러옵니다.
if os.path.exists(save_path):
    collected_keywords_data = pd.read_csv(save_path)
    
    # 데이터를 '검색어' 별로 그룹화합니다.
    grouped = collected_keywords_data.groupby('검색어')
    keyword_grouped_dfs = []  # '검색어'별 데이터프레임들을 저장할 리스트

    for keyword, group in grouped:
        # 인덱스를 리셋하고, 상위 50개 행만 선택
        top_50_group = group.reset_index(drop=True).head(50)
        
        # 결과 리스트에 추가
        keyword_grouped_dfs.append(top_50_group)
else:
    print(f"File does not exist at {save_path}. Please check the path or ensure the data collection process has been completed.")


In [7]:
print(keyword_grouped_dfs)
print(len(keyword_grouped_dfs))

[         연관키워드  월간검색수_합계      검색어
0      CMA금리비교   29490.0  CMA금리비교
1     파킹통장금리비교   56700.0  CMA금리비교
2        CMA통장   55000.0  CMA금리비교
3          CMA  145500.0  CMA금리비교
4      KB증권CMA    2990.0  CMA금리비교
5     정기예금금리비교  134600.0  CMA금리비교
6         파킹통장  402900.0  CMA금리비교
7         예금금리  192200.0  CMA금리비교
8     예금이자높은은행  119700.0  CMA금리비교
9     적금이자높은은행  120400.0  CMA금리비교
10         HTS   10530.0  CMA금리비교
11        주식시세  367190.0  CMA금리비교
12    저축은행금리비교   27510.0  CMA금리비교
13       트레이딩뷰   73500.0  CMA금리비교
14       CMA계좌   10420.0  CMA금리비교
15      예금금리비교   35500.0  CMA금리비교
16        은행금리   27220.0  CMA금리비교
17          증권  426600.0  CMA금리비교
18     IRP계좌개설   57800.0  CMA금리비교
19       ISA계좌  262000.0  CMA금리비교
20      파킹통장금리   22220.0  CMA금리비교
21       CMA금리    4700.0  CMA금리비교
22      은행금리비교   13040.0  CMA금리비교
23     삼성전자배당금   60020.0  CMA금리비교
24       수소관련주   17310.0  CMA금리비교
25    정기적금금리비교   19400.0  CMA금리비교
26      파킹통장추천   20910.0  CMA금리비교
27       주식사는법   36340.0  CMA금리비교
28       금리계산

# 4. 키워드 트렌드 및 패턴 분석

1) 정보지정

In [8]:
import utils
    # 날짜 지정
today = datetime.now(timezone('Asia/Seoul'))
day = today.strftime("%y%m%d")  

    # 저장할 폴더 미리 생성, 폴더가 이미 있으면 생성하지 않음
folders_to_create = [
        './data/tmp',
        './data/result',
        f'./data/result/{day}',
        f'./data/result/{day}/graph'
    ]

for folder in folders_to_create:
        os.makedirs(folder, exist_ok=True)

    # 분석 기간 설정
start_index = (today - relativedelta(days=1)).strftime("%Y-%m-%d")
end_index = (today - relativedelta(years=3) - relativedelta(days=1)).strftime("%Y-%m-%d")
print(['분석시작일:', start_index, '분석종료일:', end_index])

    # URI 정보 가져오기
URI = utils.get_secret("URI")

['분석시작일:', '2024-03-06', '분석종료일:', '2021-03-06']


# 5. 상세 분석

In [9]:
# 삭제할 파일 경로 정의
progress_log_path = 'progress_log.txt'  # 가정: 실제 경로에 맞게 수정해야 함
progress_state_path = 'progress_state.json'  # 가정: 실제 경로에 맞게 수정해야 함

# progress_log 파일 삭제
if os.path.exists(progress_log_path):
    os.remove(progress_log_path)
    print(f'{progress_log_path} has been deleted.')
else:
    print(f'{progress_log_path} does not exist.')

# progress_state.json 파일 삭제
if os.path.exists(progress_state_path):
    os.remove(progress_state_path)
    print(f'{progress_state_path} has been deleted.')
else:
    print(f'{progress_state_path} does not exist.')

progress_log.txt has been deleted.
progress_state.json does not exist.


trend_data : 상대지표


In [91]:
import asyncio
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
import utils
from trend import trend_maincode
import time
from pytz import timezone
import select_keyword

class analysis:
    def __init__(self):
        # 기본설정
        self.formatted_today, self.day=utils.get_today_date()
        self.state=utils.load_state()
        self.apiCallCount = self.state.get('api_request_count', 1)
        self.current_client_index = self.state.get('current_client_index', 0)
        self.keywords = utils.load_keywords('main_keyword.json')['keyword_final']
        self.request_limit = 1000
        self.keyword_index = self.state['keyword_index']
        self.standard_time = datetime.now()
        self.api_url = "https://openapi.naver.com/v1/datalab/search"
        self.today = datetime.now(timezone('Asia/Seoul'))
        self.api_request_data = pd.DataFrame(columns=['search_keywords'])
        self.start_index = (self.today - relativedelta(days=1)).strftime("%Y-%m-%d")
        self.end_index = (self.today - relativedelta(years=3) - relativedelta(days=1)).strftime("%Y-%m-%d")
        self.clients = utils.get_secret("clients")
        

        self.api_request_data_dataname = {
                'api_request_data': [],
                'keyname': []
        }

        # 데이터프레임 초기화
        self.trends_dataframes = {
        'daily_up': pd.DataFrame(),
        'weekly_up': pd.DataFrame(),
        'weekly_stay': pd.DataFrame(),
        'monthly_up': pd.DataFrame(),
        'monthly_stay': pd.DataFrame(),
        'monthly_rule': pd.DataFrame()
        }


        self.graph_tables = {
        'day': pd.DataFrame(index=pd.date_range(start=self.start_index, end=self.end_index, freq='1d')),
        'week': pd.DataFrame(index=pd.date_range(start=self.start_index, end=self.end_index, freq='7d')),
        'month': pd.DataFrame(index=pd.date_range(start=self.start_index, end=self.end_index, freq='28d'))
        }

        # 정보 저장을 위한 딕셔너리 초기화
        self.keyword_data = {
            'keyword_categories': {},
            'related_search_terms': {},
            'related_search_volume': {},
            'up_month': {}
        }
        self.review_types = {
                'daily': {
                        'function': select_keyword.select_keyword,
                        'info_dict': {},
                        'trend_df': self.trends_dataframes['daily_up'],
                        'graph_df': self.graph_tables['day'],
                        'time_period': 'daily'
                    },
            'weekly_up': {
                        'function': select_keyword.select_keyword,
                        'info_dict': {},
                        'trend_df': self.trends_dataframes['weekly_up'],
                        'graph_df': self.graph_tables['week'],
                        'time_period': 'weekly'
                    },
            'weekly_stay': {
                        'function': select_keyword.rising_keyword_analysis,
                        'info_dict': {},
                        'trend_df': self.trends_dataframes['weekly_stay'],
                        'graph_df': self.graph_tables['month'],
                        'time_period': 'weekly'
                    },
            'monthly_up': {
                        'function': select_keyword.select_keyword,
                        'info_dict': {},
                        'trend_df': self.trends_dataframes['monthly_up'],
                        'graph_df': self.graph_tables['month'],
                        'time_period': 'month'
                    },
            'monthly_stay': {
                        'function': select_keyword.rising_keyword_analysis,
                        'info_dict': {},
                        'trend_df': self.trends_dataframes['monthly_stay'],
                        'graph_df': self.graph_tables['month'],
                        'time_period': 'month'
                    },
            'monthly_rule': {
                        'function': select_keyword.monthly_rule,
                        'info_dict': {},
                        'trend_df': self.trends_dataframes['monthly_rule'],
                        'graph_df': self.graph_tables['month'],
                        'time_period': 'month'
                    }
                }
        pass


        # api 설정
    def handle_api_call(self,keywordName,df_table,currentRequestCount):
        if self.apiCallCount<= self.request_limit:

            id_num,pw,_=utils.get_client_info(self.clients,self.current_client_index)

            df_table['id']=self.clients[id_num]['client_id']
            df_table['pw']=self.clients[id_num]['client_secret']
            
            #utils.log_progress(keywordName, currentRequestCount, \
                               #len(df_table), df_table['id'].iloc[0], self.apiCallCount, self.request_limit)
            

            request_data = {
            
            'search_keywords': df_table.loc[currentRequestCount],

            }        
            return request_data, self.apiCallCount + 1, self.current_client_index
        else:
                
                # 요청 한도 초과 시 클라이언트 인덱스 업데이트
                self.current_client_index += 1
                
                self.apiCallCount=0

                if self.current_client_index >= len(self.clients):
                    print("모든 API 클라이언트의 요청 한도 초과")
                    return None, self.apiCallCount, self.current_client_index  # 처리 중단을 위한 None 반환
                else:
                    # 클라이언트 전환 후 재시도
                    return self.handle_api_call(keywordName, df_table, currentRequestCount)
                
    # 연관 검색어의 trend 데이터
    def analyze_trend_data(self, uniq):
        params = {
            "search_keywords": uniq["api_request_data"],  
            "id": self.clients['id_1']["client_id"],
            "pw": self.clients['id_1']["client_secret"],
            "api_url": self.api_url
        }
        trend_data = asyncio.run(trend_maincode(params,self.clients, self.api_url))
        return trend_data
    
    # 연관검색어의 급상승, 지속상승 로직 적용
    def select_keywords(self, trend_data):
        review_settings = {key: [] for key in self.review_types.keys()}
        for i, (index, row) in enumerate(trend_data.iterrows()):
            # 각 행의 첫 번째 열과 두 번째 열의 값 참조
            first_column_value = row['trend_data']
            second_column_value = row['keyname']
            
            for review_key, settings in self.review_types.items():
                if review_key == 'monthly_rule':
                    # monthly_rule에 대한 특별 처리
                    trend_analysis_df, graph_data_df, analysis_info, rising_month = settings['function'](first_column_value, self.day, settings['time_period'])
                    if trend_analysis_df is not None:
                        review_settings[review_key].append((trend_analysis_df, graph_data_df, analysis_info, rising_month, second_column_value,first_column_value.columns[0]))
                else:
                    # 기타 경우 처리
                    trend_analysis_df, graph_data_df, analysis_info = settings['function'](first_column_value, self.day, settings['time_period'])
                    if trend_analysis_df is not None:
                        review_settings[review_key].append((trend_analysis_df, graph_data_df, analysis_info, second_column_value,first_column_value.columns[0]))
        return review_settings
                            # tmp 랑 tmp_gph, tmp_info 얘네를 딕셔너리에 집어넣는 함수


    def create_associated_keywords_column(self,api_request_data):
        """
        Creates a new column '전체검색어' based on associated keywords grouping.

        :param api_request_data: List of dictionaries with '검색어' and '연관키워드' keys
        :return: A dictionary where each key is an 연관키워드 and its value is a concatenated string of 검색어s sharing the same 연관키워드.
        """
        # Group keywords by their associated keyword
        associated_keywords_map = {}
        for data in api_request_data:
            keyword = data['검색어']
            associated_keyword = data['연관키워드']
            if associated_keyword in associated_keywords_map:
                associated_keywords_map[associated_keyword].append(keyword)
            else:
                associated_keywords_map[associated_keyword] = [keyword]
        
        # Concatenate 검색어s for each 연관키워드
        associated_keywords_combined = {}
        for associated_keyword, keywords in associated_keywords_map.items():
            associated_keywords_combined[associated_keyword] = ', '.join(keywords)
        return associated_keywords_combined
    
    
    def remove_duplicates_by_associated_keyword(self, api_request_data):
        """
        각 연관키워드별로 중복된 검색어를 제거합니다.

        Args:
            api_request_data (List[Dict[str, Any]]): '검색어', '연관키워드' 및 기타 키를 포함하는 딕셔너리의 리스트입니다.

        Returns:
            List[Dict[str, Any]]: 연관키워드를 기준으로 중복이 제거된 새로운 딕셔너리 리스트입니다.

        각 연관키워드에 대해 첫 번째로 발견된 항목만 유지하며, 나머지 중복 항목들은 무시합니다. 만약 같은 연관키워드를 가진 검색어가 추가로 발견되면,
        해당 검색어는 첫 번째 항목의 '검색어' 값에 문자열로 추가됩니다.
        """
        unique_entries = {}
        for entry in api_request_data:
            associated_keyword = entry['연관키워드']
            keyword = entry['검색어']

            if associated_keyword not in unique_entries:
                unique_entries[associated_keyword] = entry
            else:
                if unique_entries[associated_keyword]['검색어'] != keyword:
                    unique_entries[associated_keyword]['검색어'] += f", {keyword}"

        cleaned_data = list(unique_entries.values())
        return cleaned_data

    def create_and_group_dataframe(df):
        """
        주어진 DataFrame에서 '연관키워드'와 'id' 컬럼을 선택하여 새로운 DataFrame을 생성하고,
        이를 'id' 기준으로 그룹화하는 함수입니다.

        Args:
            df (pandas.DataFrame): 원본 데이터를 포함하는 DataFrame.

        Returns:
            pandas.DataFrame: '연관키워드'와 'id' 컬럼만을 포함하는 새로운 DataFrame.
        """
        # 새 DataFrame 생성
        new_df = df[['연관키워드', 'id']].copy()
        return new_df
    # 반환 전, 호출하는
    def collect_and_analyze_keyword_trends(self,
                                            keyword_grouped_dfs: List[Any])->Dict[str,Any]:
        api_request_data_list = []  # DataFrame 대신 사용할 리스트
        total_keywords = len(keyword_grouped_dfs)

        for keywordIndex, df in enumerate(keyword_grouped_dfs, start=1):

            keywordName = df['검색어'].iloc[0]  # 데이터프레임에서 '검색어' 컬럼의 첫 번째 값을 사용

            #print(f'################################################ {keywordName} ({keywordIndex}/{total_keywords}) ################################################')
            maxKeyword = min(50, len(df))


            for currentRequestCount in range(maxKeyword):

                request_data, self.apiCallCount, self.current_client_index = self.handle_api_call(keywordName, df, currentRequestCount)




            # API 요청 예외 처리 (한도 초과 시 클라이언트 인덱스 업데이트)
                if request_data is not None:
                    api_request_data_list.append(request_data)  # 리스트에 데이터 추가
                    
                else:
                    break
                if self.current_client_index >= len(self.clients):
                    print("모든 API 클라이언트의 요청 한도 초과")
                    break
        self.keyword_index += 1

        # 리스트를 DataFrame으로 변환
            
        if api_request_data_list:
            self.api_request_data = pd.DataFrame(api_request_data_list)
            api_data_list = self.api_request_data['search_keywords'].tolist()
        
            self.api_request_data_dataname['api_request_data'] = api_data_list

    
        print(len(self.api_request_data_dataname['api_request_data']))
        uniq_api_data=self.remove_duplicates_by_associated_keyword(self.api_request_data_dataname['api_request_data'])
        print(len(uniq_api_data))
        print( uniq_api_data[3])
        print(type(uniq_api_data[3]))
        # print((self.api_request_data_dataname['api_request_data'][2000]['검색어']))
        # print("self.api_request_data_dataname['api_request_data']의 길이",len(self.api_request_data_dataname['api_request_data']))
        # self.create_associated_keywords_column(self.api_request_data_dataname['api_request_data'])

        #uniq=utils.merge_keys_for_unique_names(self.api_request_data_dataname)
        #print('uniq',uniq)

        #trend_data = pd.DataFrame({'trend_data': self.analyze_trend_data(uniq), 'keyname': uniq['keyname']})
        #return trend_data
        # 실시간 급상승 이런거 들어가는 함수부분
  

In [92]:


start=time.time()
analysis_instance = analysis()
#trend_data=analysis_instance.collect_and_analyze_keyword_trends(keyword_grouped_dfs)
trend_data = analysis_instance.collect_and_analyze_keyword_trends(keyword_grouped_dfs)

print(time.time()-start)

2250
1465
연관키워드                        CMA
월간검색수_합계                145500.0
검색어           CMA금리비교, CMA통장, 금리
id          eywsrQuPMFtlbYz1XAl7
pw                    Me4ra4s8od
Name: 3, dtype: object
<class 'pandas.core.series.Series'>
0.12550997734069824


C:\Users\chohy\AppData\Local\Temp\ipykernel_42316\992130093.py:215: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_entries[associated_keyword]['검색어'] += f", {keyword}"


### 동기 : 49.392563343048096


# 파일 정리 코드


make_directory('./data')
make_directory('./data/rl_srch')
make_directory(f'./data/rl_srch/{day}')  # 키워드별 연관검색어 리스트 저장

이걸로 생성된 파일 삭제하는 코드

# 로그파일 삭제

1. 세션이 끊겼을 때 어떻게해야 좋을지 판단 로그파일을 인터넷에 깔아서 관리할까? 아니면 딕셔너리로 보이게 해서 관리할까? 라는 생각
2. 너무 하드코딩되어있는 부분 존재 이런부분은 처리하기
3. missing데이터 같은 경우 딕셔너리로 관리해서 daily나 weekly일때 다르게 구분하기 편하게 하기
4. 로그파일로 따로 관리하는거 좋은가? 에 대한 생각해보기
5. 주석같은거 중요한거마다 하기
6. 